# **Artificial Neural Network in R for Regression**

## **Data Set Information:**
The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP) of the plant.

**Attribute Information:**

Features consist of hourly average ambient variables
* Temperature (T) in the range 1.81°C and 37.11°C,
* Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
* Relative Humidity (RH) in the range 25.56% to 100.16%
* Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
* Net hourly electrical energy output (EP) 420.26-495.76 MW

## **Part - 1: Data Preprocessing**

### **Importing the dataset**

In [1]:
ds = read.csv('/content/Power Plant Data.csv')
head(ds)

,AT,V,AP,RH,PE
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14.96,41.76,1024.07,73.17,463.26
2,25.18,62.96,1020.04,59.08,444.37
3,5.11,39.40,1012.16,92.14,488.56
4,20.86,57.32,1010.24,76.64,446.48
5,10.82,37.50,1009.23,96.62,473.90
6,26.27,59.44,1012.23,58.77,443.67


In [2]:
summary(ds)

       AT              V               AP               RH        
 Min.   : 1.81   Min.   :25.36   Min.   : 992.9   Min.   : 25.56  
 1st Qu.:13.51   1st Qu.:41.74   1st Qu.:1009.1   1st Qu.: 63.33  
 Median :20.34   Median :52.08   Median :1012.9   Median : 74.97  
 Mean   :19.65   Mean   :54.31   Mean   :1013.3   Mean   : 73.31  
 3rd Qu.:25.72   3rd Qu.:66.54   3rd Qu.:1017.3   3rd Qu.: 84.83  
 Max.   :37.11   Max.   :81.56   Max.   :1033.3   Max.   :100.16  
       PE       
 Min.   :420.3  
 1st Qu.:439.8  
 Median :451.6  
 Mean   :454.4  
 3rd Qu.:468.4  
 Max.   :495.8  

In [3]:
#find number of rows with missing values
sum(!complete.cases(ds))

[1] 0

### **Splitting the dataset into the Training set and Test set**

In [4]:
install.packages('caTools')
library(caTools)
set.seed(123)
split = sample.split(ds$PE, SplitRatio = 4/5)
head(split)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



[1]  TRUE  TRUE  TRUE FALSE FALSE  TRUE

In [5]:
train_set = subset(ds, split == TRUE)
test_set = subset(ds, split == FALSE)
print(head(train_set,5))

print(head(test_set,3))

     AT     V      AP    RH     PE
1 14.96 41.76 1024.07 73.17 463.26
2 25.18 62.96 1020.04 59.08 444.37
3  5.11 39.40 1012.16 92.14 488.56
6 26.27 59.44 1012.23 58.77 443.67
7 15.89 43.96 1014.02 75.24 467.35
     AT     V      AP    RH     PE
4 20.86 57.32 1010.24 76.64 446.48
5 10.82 37.50 1009.23 96.62 473.90
8  9.48 44.71 1019.12 66.43 478.42


## **Part 2 - Building & Training the ANN**

In [6]:
install.packages("h2o")
library(h2o)
h2o.init(nthreads = -1)
model = h2o.deeplearning(y = 'PE',
                         training_frame = as.h2o(train_set),
                         activation = 'Rectifier',
                         hidden = c(5,5),
                         epochs = 100,
                         stopping_metric="MSE", ## could be "misclassification","logloss","r2"
                         train_samples_per_iteration = -2)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RCurl’



----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit https://docs.h2o.ai

----------------------------------------------------------------------



Attaching package: ‘h2o’


The following objects are masked from ‘package:stats’:

    cor, sd, var


The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc





H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpbrGEff/file1257ca93818/h2o_UnknownUser_started_from_r.out
    /tmp/RtmpbrGEff/file1253ecb9e8f/h2o_UnknownUser_started_from_r.err


Starting H2O JVM and connecting: .... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 seconds 764 milliseconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.36.0.3 
    H2O cluster version age:    1 month and 4 days  
    H2O cluster name:           H2O_started_from_R_root_ppf172 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   3.17 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:   

## **Part 3 - Making the predictions and evaluating the model**

### **Predicting the Test set results**

In [7]:
y_pred = h2o.predict(model, newdata = as.h2o(test_set[-5]))

y_pred = as.data.frame(y_pred)


  |======================================================================| 100%
  |======================================================================| 100%


In [8]:
summary(model)

Model Details:

H2ORegressionModel: deeplearning
Model Key:  DeepLearning_model_R_1647874682704_1 
Status of Neuron Layers: predicting PE, regression, gaussian distribution, Quadratic loss, 61 weights/biases, 4.4 KB, 765,400 training samples, mini-batch size 1
  layer units      type dropout       l1       l2 mean_rate rate_rms momentum
1     1     4     Input  0.00 %       NA       NA        NA       NA       NA
2     2     5 Rectifier  0.00 % 0.000000 0.000000  0.001753 0.000660 0.000000
3     3     5 Rectifier  0.00 % 0.000000 0.000000  0.016275 0.044116 0.000000
4     4     1    Linear      NA 0.000000 0.000000  0.001900 0.000975 0.000000
  mean_weight weight_rms mean_bias bias_rms
1          NA         NA        NA       NA
2   -0.133833   0.401025  0.514058 0.586826
3   -0.200495   0.803140  0.832801 0.288028
4   -0.203933   0.584445  0.103761 0.000000

H2ORegressionMetrics: deeplearning
** Reported on training data. **
** Metrics reported on full training frame **

MSE:  17.1261

In [9]:
y_test = as.data.frame(test_set$PE)

### **Metrics**

In [10]:
d = y_pred - y_test
head(d)

,predict
,<dbl>
1,1.32983971
2,-0.08426849
3,1.64182828
4,4.85579652
5,-3.95473218
6,6.07932437


In [11]:
d = as.double(unlist(d))
d

[1]   1.329839711  -0.084268495   1.641828284   4.855796524  -3.954732180
   [6]   6.079324371  -8.287641547  -1.762351112   2.803007899   2.793828093
  [11]  -4.350576656   5.031828548   2.563819932  -5.950539207   2.669190918
  [16]  -4.373820140   7.380992116  -2.771670076   2.754092476  -6.317342750
  [21]  -3.955402717   3.140843856   0.070434686  -1.650338793  -2.758695373
  [26]   2.724940105   5.786088502   0.974550646  -3.851701533   0.460335169
  [31]   2.746832111   1.635995208   0.248033891   1.500025832   0.137010925
  [36]  -2.787981655   0.359860915   5.629973256   1.018753803   4.593336367
  [41]   4.262934303   0.150825287   1.440551024  -2.286227704  -1.269380417
  [46]  -7.693016270   1.674160011  -0.918281774  -3.911617678  -0.231392223
  [51]   4.768172691  -1.928713601   3.354429893  -0.086871239  -2.677233245
  [56]   1.009748207  -3.295654814  -5.058502962  -8.163791188  -1.523872328
  [61]   0.540567199  -0.794069975  -1.738396712  -5.038309502   3.547629119
  [66]  -3.320490862  -3.165982985   6.216297774  10.550786286   9.764789780
  [71]  -0.188369031   0.763447111  -0.995934864  -1.232211807   2.028288845
  [76]   0.839667734  -4.820919108  -4.521932669  -2.498653969  -2.965338542
  [81]  -3.067627177   0.831607294  -5.366762302   2.022216633  -2.795607991
  [86]  -3.477542888  -4.762662381   2.050318141   0.334536611   1.965630267
  [91]  -4.017805224   7.141985065  -0.698719892   8.162535297   3.481859222
  [96]  -0.816627771   1.773727470   2.981855047  -3.878886469  -3.067261732
 [101]   6.063863318  -3.770637465   2.078237536  -0.495168699   1.902182426
 [106]   2.480720334   1.707075406 -11.286773010   5.444055643   0.468027403
 [111]  -2.140780739  -0.408229113  -5.453607772  -9.837505084  -3.917388045
 [116]  -2.870771776  -0.188615649   6.313766388  -4.106066798  -2.242290225
 [121]   5.774961725  -4.641212346   2.730082522   3.047094888   6.658094122
 [126]  -1.352968987   2.467814100  -0.427007464  -5.296078522  -2.139595465
 [131]  -2.921785123  -2.692863095  -3.839296962  11.514561235  -7.062517056
 [136]  -5.705338656  -7.781354502  -3.813697996   1.846069224   0.799747878
 [141]  -0.529767489  -2.717515318   2.712014873   3.853387013  -0.990280832
 [146]   2.137509748  -4.900841403  -1.106538684   3.596438261  -7.559072283
 [151]  -7.311984269  -1.103082232  -4.935923716   0.248752320  -4.266913319
 [156]  -0.533782128  -1.375807580   4.665045710  -1.214009464   4.043998750
 [161]   1.720058694   1.405217834  -1.435475088   1.727493883  -2.350536552
 [166]  -2.469021499  -2.114083641   3.882701435   1.180674275   2.684665666
 [171]  -5.312066552  -3.051701935   5.248045164   3.123478395   0.977437085
 [176]  -0.486208612   0.237420567  -1.767251136   7.126592699  -2.921544112
 [181]   0.333200945   3.506189441  -4.788670849  -2.713354521   0.929407870
 [186]   3.755767170 -11.985082785   2.509272043   1.237108694  -1.889304797
 [191]  -1.123020181   0.548614195  -9.429523482  -2.868832049   6.600055948
 [196]   6.325047088  -0.890849036  -2.568905973   3.193941560   1.176608445
 [201]   2.321217717  -5.964475836  -1.076193289  -3.191386921   3.039573686
 [206]  -0.418522574   3.174259965  -5.212754443   4.676437493  -3.845607045
 [211]   1.055055660  -4.578894078  -2.343581599   0.096684647   0.179654576
 [216]   3.556815901  -0.556095133  -5.394384870  -0.655531208   1.059160442
 [221]  -1.763970534  -5.395033257  -4.548018065   1.236328499  -2.820881578
 [226]   1.919481510  -0.865300612  -7.880085796   1.751746130   8.297074122
 [231]   6.793369730   4.160192496  -2.108722708  -0.325976551 -10.632124223
 [236]  -2.699637730   0.467301457  -3.728375161   0.045638244  -1.208541051
 [241]  -6.459864386  -6.516477241   4.556238469  -2.369357685   1.310610449
 [246]  10.958832679  -8.012823833  -0.858589495   4.155439744  -7.519029478
 [251]  -2.727259407  -1.930533162  -6.946846521  -3.909181557   2.905370177
 [256]   2.248003966   0.647277621   2.272627927  -3.254896262  -2.754526156

In [12]:
mse = mean((d)^2)
mae = mean(abs(d))
rmse = sqrt(mse)
R2 = 1-(sum((d)^2)/sum((test_set$PE-mean(test_set$PE))^2))

cat(" MAE:", mae, "\n", "MSE:", mse, "\n", 
    "RMSE:", rmse, "\n", "R-squared:", R2)

 MAE: 3.170411 
 MSE: 17.36656 
 RMSE: 4.16732 
 R-squared: 0.939426

In [16]:
#install.packages('hydroGOF')
library(hydroGOF)
 mae = mae(y_pred, test_set$PE)
mse = mse(y_pred, test_set$PE)
#Calculate RMSE 
RMSE=rmse(y_pred,test_set$PE)
cat(" MAE:", mae, "\n", "MSE:", mse, "\n", 
    "RMSE:", rmse, "\n")

 MAE: 3.170411 
 MSE: 17.36656 
 RMSE: 4.16732 


In [17]:
#Correlation, Variance and Covariance
Correlation = cor(y_pred, test_set$PE)
Covariance = cov(y_pred, test_set$PE)
Variance = var(y_pred, test_set$PE)
cat("Correlation:", Correlation, "\n", "Covariance:", Covariance, "\n", 
    "Variance:", Variance, "\n")

Correlation: 0.9700344 
 Covariance: 278.987 
 Variance: 278.987 


In [18]:
h2o.shutdown()

Are you sure you want to shutdown the H2O instance running at http://localhost:54321/ (Y/N)? Y
